In [18]:
import numpy as np
import pandas as pd

In [19]:
df =pd.read_csv('HousingData.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [20]:
df = df.dropna()
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0.17783,0.0,9.69,0.0,0.585,5.569,73.5,2.3999,6,391,19.2,395.77,15.10,17.5
500,0.22438,0.0,9.69,0.0,0.585,6.027,79.7,2.4982,6,391,19.2,396.90,14.33,16.8
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9


In [21]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# a. Regression with different algo

In [23]:
estimators = [
    ('lr',LinearRegression()),
    ('dt',DecisionTreeRegressor()),
    ('svr',SVR())
]

# Check every estimator validation

In [24]:
for estimator in estimators:
    scores = cross_val_score(estimator[1],X,y,cv=10,scoring='r2')
    print(estimator[0],np.round(np.mean(scores),2))

lr 0.29
dt 0.24
svr -0.37


# 1. Voting regressor

In [25]:
from sklearn.ensemble import VotingRegressor
voting_regressor = VotingRegressor(estimators=estimators)
np.mean(cross_val_score(voting_regressor,X,y,scoring='r2',cv=10))

0.3636295644331763

# 2. Wighted Voting

In [26]:
def weightedVotingRegression(max_weight):
    maxAcc = 0
    im = 0
    jm = 0
    km = 0
    for i in range(1,max_weight+1):
        for j in range(1,max_weight+1):
            for k in range(1,max_weight+1):
                weighted_vc = VotingRegressor(estimators=estimators,weights=[i,j,k])
                x = np.mean(cross_val_score(weighted_vc,X,y,cv=10,scoring='r2'))
                if x > maxAcc:
                    maxAcc = x
                    im = i
                    jm = j
                    km = k
                print(f"for i={i}, j={j}, k={k} acc={np.round(x,2)}")

    return maxAcc, im, jm, km           

In [27]:
acc, i ,j ,k = weightedVotingRegression(4)

for i=1, j=1, k=1 acc=0.42
for i=1, j=1, k=2 acc=0.28
for i=1, j=1, k=3 acc=0.19
for i=1, j=1, k=4 acc=0.12
for i=1, j=2, k=1 acc=0.38
for i=1, j=2, k=2 acc=0.35
for i=1, j=2, k=3 acc=0.26
for i=1, j=2, k=4 acc=0.22
for i=1, j=3, k=1 acc=0.4
for i=1, j=3, k=2 acc=0.31
for i=1, j=3, k=3 acc=0.29
for i=1, j=3, k=4 acc=0.25
for i=1, j=4, k=1 acc=0.39
for i=1, j=4, k=2 acc=0.33
for i=1, j=4, k=3 acc=0.31
for i=1, j=4, k=4 acc=0.27
for i=2, j=1, k=1 acc=0.44
for i=2, j=1, k=2 acc=0.36
for i=2, j=1, k=3 acc=0.3
for i=2, j=1, k=4 acc=0.22
for i=2, j=2, k=1 acc=0.43
for i=2, j=2, k=2 acc=0.39
for i=2, j=2, k=3 acc=0.33
for i=2, j=2, k=4 acc=0.28
for i=2, j=3, k=1 acc=0.41
for i=2, j=3, k=2 acc=0.41
for i=2, j=3, k=3 acc=0.37
for i=2, j=3, k=4 acc=0.32
for i=2, j=4, k=1 acc=0.42
for i=2, j=4, k=2 acc=0.38
for i=2, j=4, k=3 acc=0.38
for i=2, j=4, k=4 acc=0.34
for i=3, j=1, k=1 acc=0.44
for i=3, j=1, k=2 acc=0.39
for i=3, j=1, k=3 acc=0.34
for i=3, j=1, k=4 acc=0.28
for i=3, j=2, k=1 acc=0.44
for

In [28]:
acc, i, j, k

(0.5029134178111319, 4, 3, 1)

# a. Regression with same algo

In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
estimators = [
    ('dt1',DecisionTreeRegressor(max_depth=1)),
    ('dt2',DecisionTreeRegressor(max_depth=3)),
    ('dt3',DecisionTreeRegressor(max_depth=5)),
    ('dt4',DecisionTreeRegressor(max_depth=7)),
    ('dt5',DecisionTreeRegressor(max_depth=None)),    
]

In [31]:
for estimator in estimators:
    scores = cross_val_score(estimator[1],X,y,cv=10,scoring='r2')
    print(estimator[0],np.round(np.mean(scores),2))

dt1 -0.91
dt2 -0.04
dt3 0.32
dt4 0.29
dt5 0.24


In [32]:
voting_regressor_sameAlgo = VotingRegressor(estimators=estimators)
np.mean(cross_val_score(voting_regressor_sameAlgo,X,y,scoring='r2',cv=10))

0.28005036331484406